Khai báo các thư viện cần thiết

In [5]:
import numpy as np
import cv2
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from skimage.transform import resize

Đọc dữ liệu

In [6]:
path = 'D:/Code/python/mat3533/practice05/data/face_data/'
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
'noglasses', 'normal', 'rightlight','sad',
'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png
# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)
h = im.shape[0] # hight
w = im.shape[1] # width
D = h * w
N = len(states)*15
print(N, D, h, w)
X = np.zeros((D, N))
# collect all data
count = 0
# there are 15 people
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1
y = np.repeat(range(1, 16), len(states))

165 77760 243 320


Giảm số chiều về 5 chiều

In [8]:
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X.T)

Phân chia dữ  liệu theo tỉ lệ train:test=0.7:0.3

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

Chạy mô hình hồi quy Logistic nhiều chiều và in ra độ chính xác

In [10]:
from sklearn.linear_model import LogisticRegression

model_log = LogisticRegression()
model_log.fit(X_train, y_train)

y_pred_log = model_log.predict(X_test)
print(f"Prediction: \n{y_pred_log}")

accuracy_log = accuracy_score(y_test, y_pred_log)
print("Accuracy: ", accuracy_log)

Prediction: 
[13 11 12  6  9  7 15  5 10 14  2  9  2  8  7  7 13 10 14  2  7  2  4  1
  9  7 14  6 13  7 13  7  6  7  1 13  5  4  3  8  7 13 11  9 13  2  7 10
  5  7]
Accuracy:  0.6


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Chạy mô hình Naive Bayes và in ra độ chính xác

In [11]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

y_pred_nb = model_nb.predict(X_test)
print(f"Prediction: \n{y_pred_nb}")

accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Accuracy: ", accuracy_nb)

Prediction: 
[15 11 12 14  9  7 15  5 10 14  2  9  2 10  7  7 13  4 14  2  7  2  4  1
  9  8 14  6 13  8 13  7  6  9  1  3  5  4  3  8  7 13 11  9 13  1  7  5
  5  8]
Accuracy:  0.52


Đọc dữ liệu Test và đưa về cùng kích thước như trong ảnh dữ liệu (H = 320; W = 243)

In [12]:
path = 'D:/Code/python/mat3533/practice05/data/face_test_data/'
states = ['rightlight','sad', 'sleepy', 'surprised', 'wink']
prefix = 'picture'
surfix = '.png' #file extension is png
# open one picture to get the image's size
fn = prefix + surfix
im = cv2.imread(path + fn, 0)
N = 5
h = 320 # hight
w = 243 # width
D = h * w
print(N, D, h, w)
X_test_new = np.zeros((D, N))
for i in range(5):
    X_test_new[:, i] = resize(X_test_new[:, i].reshape((h, w)), (320, 243)).reshape(-1)
# collect all data
count = 0
for state in states:
    # get name of each image file
    fn = path + prefix + '_' + state + surfix
    # open the file and read as grey image
    tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
    # then add image to dataset X
    X_test_new[:, count] = tmp.reshape(D)
    count += 1

5 77760 320 243


Giảm số chiều cho dữ liệu test về 5 chiều

In [18]:
pca_new = PCA(n_components = 5)
X_test_new_pca = pca_new.fit_transform(X_test_new.T)

Dự đoán dữ liệu Test với mô hình hồi quy logistic nhiều lớp đã chạy

In [20]:
y_pred_new_log = model_log.predict(X_test_new_pca)
print(y_pred_new_log)

[14  2  9 10  1]


Dự đoán dữ liệu Test với mô hình Naive Bayes đã chạy

In [21]:
y_pred_new_nb = model_log.predict(X_test_new_pca)
print(y_pred_new_nb)

[14  2  9 10  1]
